In [3]:
import re
import string
import numpy as np
import pandas as pd
import nltk
from itertools import islice

In [101]:
df = pd.read_excel('./data/facebook-bot.xlsx')

In [102]:
df

,timestamp (UTC),first name,last name,gender,text
0,2020-12-22 17:05:31,Mouad,Boulahdoud,male,اسير في تقود اولد القحبة
1,2020-12-22 17:07:17,Hassàn,Ait Baha,male,زامل لي كالها ليك
2,2020-12-22 17:23:14,Mouad,Boulahdoud,male,یا شیوعي
3,2020-12-22 17:36:55,Mouad,Boulahdoud,male,نشدك نحويك
4,2020-12-22 17:37:02,Mouad,Boulahdoud,male,عارفك ولد القحبة
...,...,...,...,...,...
210,NaT,NaN,NaN,NaN,صهيوني
211,NaT,NaN,NaN,NaN,سكايري
212,NaT,NaN,NaN,NaN,شمكار
213,NaT,NaN,NaN,NaN,مصيصة


In [103]:
df['text'].isnull().sum()

0

In [104]:
text = ''
for sentence  in df['text']:
    text += ' ' + sentence

In [225]:
text

' اسير في تقود اولد القحبة زامل لي كالها ليك یا شیوعي نشدك نحويك عارفك ولد القحبة تقود نتا قواد يا ارهابي بلاصتك الكوزينة يا زنديق تفو خراا الله ينعل ليك الواليدين ا الحمار الله ينعل طواسلك يا القواد يا الزامل يا ولد القحبة غانشدك نحويك فاكيوو يا الكلب يا حمار يا كلب ولد القحبة طبون يماك زليجة دينموا كيدار بغل زامل زيادة اللور كوزينتك تريكة ربعمية مكمشة قلاوي زب راس القلوة وزبي وراس الطاروا  والحمار  والكلب  والقلاوي والملحد وولد العاهرة  ولد الباغية  وجغل  الله انعل لمك لولدين فهاد النهار السعيد بقات فاللحية سير الله انعل لمك مها الله انعل الكلبة تاع بات بات امك ياصفيرو يابوخنونة كنت ايهودي بلفعايل دبا وليتي ايهودي بلوراق ياالخانزة بنت العاهرة يالبايرة وا لقلاوي غنحويك غانضرب ليك سوا ديالك ههه المكلخ لاخور واش نتى حمار ترجل معانا فعايل العيالات خونتا لاخور ولد العبد العاهرة تقحبين الزامل العطاي عطوة ماشي فعايل ديال الرجال قزحي سير تقور انحوي مك العزي لاخور وجه السوى واش نتى متخلف طالع عندك راس الزب وجه القلوة كيداير قحبة نتا اصلا ككيدار وجوه الزب باقي قديم ترمتك مصدية يالزامل سير الله

In [3]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

In [4]:
stop_words = nltk.corpus.stopwords.words('arabic')

In [5]:
# function to clean and normalize text 
def clean_text(text):
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

In [182]:
cleaned_text = clean_text(text)

In [190]:
vocab = []
token_temp = cleaned_text.split() # create tokens
for i in range(len(token_temp)):
    if token_temp[i] not in vocab: # to check if word is already in vocab
        vocab.append(token_temp[i])

In [9]:
vocab =['اسير',
 'تقود',
 'اولد',
 'القحبه',
 'زامل',
 'شیوعي',
 'نشدك',
 'نحويك',
 'قواد',
 'ارهابي',
 'الكوزينه',
 'زنديق',
 'تفو',
 'خرا',
 'ينعل',
 'الحمار',
 'طواسلك',
 'القواد',
 'الزامل',
 'ياولد',
 'غانشدك',
 'فاكيو',
 'الكلب',
 'حمار',
 'كلب',
 'طبون',
 'يماك',
 'زليجه',
 'دينمو',
 'كيدار',
 'بغل',
 'زياده',
 'اللور',
 'كوزينتك',
 'تريكه',
 'ربعميه',
 'قلاوي',
 'زب',
 'راس',
 'القلوه',
 'وزبي',
 'وراس',
 'الطارو',
 'الحمار',
 'والكلب',
 'القلاوي',
 'الملحد',
 'العاهره',
 'الباغيه',
 'وجغل',
 'انعل',
 'لمك',
 'اللحيه',
 'سير',
 'مها',
 'الكلبه',
 'بات',
 'امك',
 'ياصفيرو',
 'يابوخنونه',
 'يهودي',
 'يالخانزه',
 'بنت',
 'يالبايره',
 'لقلاوي',
 'غنحويك',
 'غانضرب',
 'سوا',
 'ديالك',
 'المكلخ',
 'لاخور',
 'حمار',
 'فعايل',
 'العيالات',
 'خونتا',
 'العبد',
 'تقحبين',
 'الزامل',
 'العطاي',
 'عطوه',
 'قزحي',
 'تقور',
 'انحوي',
 'العزي',
 'وجه',
 'السوي',
 'متخلف',
 'طالع',
 'الزب',
 'كيدار',
 'قحبه',
 'ككيدار',
 'قديم',
 'ترمتك',
 'مصديه',
 'يالزامل',
 'لينعل',
 'بوك',
 'المرض',
 'زوامل',
 'طارو',
 'يحويك',
 'دين',
 'الزاكوري',
 'الصحري',
 'خايب',
 'كتخري',
 'عنيبا',
 'كتخور',
 'المعاق',
 'كتشرب',
 'بول',
 'البعير',
 'ياالبهيمه',
 'زيدها',
 'فكرك',
 'منغولي',
 'العروبي',
 'فرك',
 'المنغوليان',
 'مراه',
 'سوق',
 'مالك',
 'الزك',
 'كحل',
 'الواسعه',
 'ولدالهاربه',
 'حتون',
 'ختك',
 'الكلب',
 'الطاحونه',
 'عزوي',
 'دزب',
 'لقحبه',
 'انعن',
 'طابون',
 'اكبر',
 'نك',
 'كس',
 'اداك',
 'مال',
 'دينمك',
 'يهودي',
 'الديوث',
 'عزي',
 'تامل',
 'غليض',
 'تب',
 'يالكحلوش',
 'قزام',
 'اغيول',
 'بقره',
 'لفعه',
 'ميكروب',
 'دخش',
 'دحش',
 'جحش',
 'خانز',
 'البيطان',
 'القحاب',
 'نيك',
 'ملحد',
 'كافر',
 'زنديق',
 'العزوه',
 'عشير',
 'الضلمه',
 'الشكلاط',
 'هبيل',
 'ومزنزن',
 'زبي',
 '🖕',
 '👎',
 '💩',
 'الدفن',
 'خريتي',
 'حامض',
 'زويمل',
 'النخبه',
 'الخرا',
 'مقوده',
 'مزمول',
 'منافق',
 'واكلاك',
 'اسكتشا',
 'قود',
 'عليا',
 'تاعت',
 'المتقوبه',
 'وباراكا',
 'ازبي',
 'فرعتيلي',
 'كري',
 'القواده',
 'معاق',
 'فراسك',
 'اختك',
 'يعطيك',
 'العداب',
 'مرض',
 'يسيس',
 'ياكل',
 'الحلوي',
 'القاف',
 'تمصيه',
 'داعشيه',
 'كسمك',
 'عياش',
 'الزابه',
 'هربه',
 'مساطه',
 'ربك',
 'كفات',
 'صهيوني',
 'سكايري',
 'شمكار',
 'مصيصه',
 'خرزه',
 'أبازا',
 'أسهولز',
 'أويلك',
 'الويل',
 'إبشي',
 'كسمك',
 'الله إنعل',
 'إيخان',
 'ابن القحبة',
 'طبون',
 'البرتوش',
 'النكح',
 'البزولة',
'البزولة',
'البزول',
'البعابع',
'الخصيتين'
'الخصية',
'البوسان',
'بوسان',
'التب',
'الترمة',
'ترمة',
'التقحبين',
'تتقحبن',
'التكفات',
'تكفات',
'الثب',
'ثب',
'الحنكليس',
'حنكليس',
'الحوا',
'حوا',
'الخنز',
'خانز',
'خنز',
'الزامل',
'زامل',
'نعلديلمكم',
'ازبي',
'زبي',
'الزب',
'زب',
'الزك',
'زك',
'الزمر',
'زمر',
'تزمر',
'الزملة',
'زملة'
'زمل',
'الشلح',
'شلح',
'الطبون',
'طبون',
'العربوز',
'عربوز',
'العيان',
'عيان',
'الغنزوز',
'غنزوز',
'الفشلوق',
'فشلوق',
'القحبة',
'قحبة',
'قلاوي',
'القواويد',
'قواويد',
'قواد',
'الليحرق',
'ليحرق',
'المخري',
'الخراء',
'خراء',
'المزبب',
'مزبب',
'باطرونة',
'برتوش',
'قلوة',
'القلوة',
'فكري',
'تقتلك',
'تحوى',
'ترماحة',
'تشهيتك',
'مقاودة',
'مقاودا',
'تناكت',
'تشهيتك',
'موسخ',
'الوسخ',
'الزبل',
'ثامل',
'جلاخة',
'حازق',
'حلوف',
'حليوة',
'حلاب',
'عنيبة',
'حويتك',
'حويني',
'حوي',
'خايب',
'خايبة',
'المجرم',
'مجرم',
'لاعدام',
'اعدام',
'الاعدام',
'كلاب',
'شلاهبي',
'محساد',
'حسد',
'العفن',
'الوسخ',
'دالخرا',
'زبل',
'دحش',
'قتل',
'غتاصب',
'شماتة',
'طبنمك',
'طبون',
'عزوة',
'كحلوش,
'كاكا'
]

In [10]:
off_vocab = df = pd.DataFrame(vocab) 
off_vocab.head()

,0
0,اسير
1,تقود
2,اولد
3,القحبه
4,زامل


In [8]:
off_vocab.to_csv('./data/off_vocab.csv', index=False, header=False, encoding="utf-16")

In [33]:
import io
import csv
with io.open('./data/off_vocab.csv', 'w', newline='', encoding="utf-16") as file:
        writer = csv.writer(file, delimiter =",", quoting=csv.QUOTE_ALL)
        for v in vocab:
            writer.writerow([v])

In [11]:
off_vocab

,0
0,اسير
1,تقود
2,اولد
3,القحبه
4,زامل
5,شیوعي
6,نشدك
7,نحويك
8,قواد
9,ارهابي
